In [2]:
# importing function from a different ipny 
import pandas as pd
import collections
from keras.initializers import Constant
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import operator
from itertools import product
import sys
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
#https://www.amazon.com/Neural-Networks-Deep-Learning-Textbook/dp/3319944622/ref=cm_cr_arp_d_product_top?ie=UTF8
#https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6

/home/titli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000

In [4]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[:10000], ham_index[:9997]), axis=0)
keras_data_new = keras_data.iloc[new_index]
labels = keras_data_new.isspam
keras_data_new.head()

,Subject,isspam
0,"Confidential :Soma:, Ci@lis, :P:ntermin",1
1,¢Â ÇªÁüÇÑ ¼³³¯ ÀÌº¥Æ® ÀÀ¸ðÇØ¼­ ºÎ¸ð´Ô²² ¼±¹°ÇÏ...,1
2,_na_,1
3,StOck 0ppurtunities - their sh0Oting,1
6,All your prescr[iption needs right here,1


In [5]:
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

In [6]:
embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Found 400000 word vectors.
Processing text dataset


In [7]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_new.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]



Found 15425 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 2)


In [8]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')



Preparing embedding matrix.
Training model.


# Embedding Model

In [9]:
keras_data = pd.read_csv('subject_spam.csv', index_col=0, encoding='utf8', engine='python')
keras_data.fillna("_na_", inplace = True)
keras_data = keras_data.drop(keras_data[keras_data.isspam == "_na_"].index)
spammer = {'spam ': 1,'ham ': 0} 
keras_data.isspam = [spammer[item] for item in keras_data.isspam] 
spam_index = keras_data[keras_data.isspam ==1].index
ham_index = keras_data[keras_data.isspam == 0].index
new_index = np.concatenate((spam_index[10000:19000], ham_index[9997:19000]), axis=0)
keras_data_test_set = keras_data.iloc[new_index]
labels_test_set = keras_data_test_set.isspam
keras_data_test_set.head()

,Subject,isspam
19223,RE: Daily Summary of Risk Data,0
19224,Softwares CDS all software under 15$ and 99$!,1
19226,Hotel Room Bargains at up to 70% off! Save in...,1
19227,RE: Greeley Gas Company,0
19228,"Any Software just in under 15-99$, Xp-adobe etc",1


In [10]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(keras_data_new.Subject)
sequences = tokenizer.texts_to_sequences(keras_data_test_set.Subject)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_test_set = to_categorical(np.asarray(labels_test_set))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
#labels_test_set = labels_test_set[indices]
x_test = data

Found 15425 unique tokens.
Shape of data tensor: (17088, 1000)
Shape of label tensor: (19997, 2)


In [13]:
embedding1 = pd.read_csv('embeddings_models1.csv')
embedding = pd.read_csv('embeddings_models.csv')

In [14]:
result = pd.concat([embedding1, embedding], axis=0)
result_sort = result.sort_values(['val_loss'])
result_sort.head()

,dl,fl1,fl2,fl3,kl,layer,optimizer,train_acc,train_loss,val_acc,val_loss,trainable_params
297,16,32,32,64,5,1,Nadam,0.909676,0.206020,0.897224,0.242720,16594
359,32,32,32,128,5,3,Nadam,0.910114,0.212017,0.899725,0.243949,45986
47,128,32,16,16,5,3,Nadam,0.905988,0.218607,0.901975,0.245111,22338
399,32,32,64,16,5,1,Adam,0.909301,0.206974,0.897974,0.245414,17154
107,16,32,16,64,5,3,Nadam,0.909551,0.212853,0.897974,0.248456,24866


In [26]:
def embeddings(fl1=32, fl2=32, fl3=64, dl=16, optimizer= 'RMSprop', kl = 5, layer =1 ):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    if (layer == 1):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
    elif (layer == 2):
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        
    else:
        x = Conv1D(filters = fl1, kernel_size = kl, activation='relu')(embedded_sequences)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl2, kernel_size = kl, activation='relu')(x)
        x = MaxPooling1D(pool_size = kl)(x)
        x = Conv1D(filters = fl3, kernel_size = kl, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(units = dl, activation='relu')(x)
    preds = Dense(2, activation='softmax')(x)
    model = Model(sequence_input, preds)
    model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizer,
              metrics=['acc'])
   
    return model

## 1st model

In [16]:
kwargs = dict(fl1=32, fl2= 0, fl3=0, kl=5, dl=64, optimizer= ''.join('Nadam'), layer=1)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model1_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 66s 4ms/step - loss: 0.3466 - acc: 0.8469 - val_loss: 0.2931 - val_acc: 0.8747
Epoch 2/2
17088/17088 [==============================] - 29s 2ms/step


## 2nd model

In [17]:
kwargs = dict(fl1=32, fl2= 32, fl3=128, kl=5,dl=32, optimizer= ''.join('Nadam'), layer=3)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model2_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 71s 4ms/step - loss: 0.3468 - acc: 0.8453 - val_loss: 0.3535 - val_acc: 0.8555
Epoch 2/2
17088/17088 [==============================] - 32s 2ms/step


## 3rd model

In [27]:
kwargs = dict(fl1=32, fl2= 16, fl3=16, kl=5, dl=128, optimizer= ''.join('Nadam'), layer=3)
model = embeddings(**kwargs)
model.fit(x_train, y_train, batch_size= 16, epochs=2, validation_data=(x_val, y_val))
model3_test_y = model.predict(x_test, batch_size=16, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 68s 4ms/step - loss: 0.3494 - acc: 0.8434 - val_loss: 0.3224 - val_acc: 0.8630
Epoch 2/2
17088/17088 [==============================] - 31s 2ms/step


In [19]:
pred_test_y = 0.33*model1_test_y + 0.33*model2_test_y + 0.34*model3_test_y
A = pred_test_y[:,0]
B = pred_test_y[:,1]
pred_array =[]

for i in range(len(pred_test_y)):
    if (A[i]>0.99):
        pred_array.append(0)
    else:
        pred_array.append(1)

In [20]:
predictions = pred_array
actuals = labels_test_set.values
true_pos= 0
true_neg = 0
false_pos = 0
false_neg = 0
for i in range (17088):
    if ((predictions[i]==1) & (actuals[i]==1)):
        true_pos = true_pos+1
    elif((predictions[i]==0) & (actuals[i]==0)):
        true_neg = true_neg+1
    elif((predictions[i]==1) & (actuals[i]==0)):
        false_pos = false_pos +1
    elif((predictions[i]==0) & (actuals[i]==1)):
        false_neg = false_neg+1
prec=true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
F1=2*(prec*recall/(prec+recall))
print('F1-score on test set is {0:.2f}%'.format(round(F1*100),2))

F1-score on test set is 66.00%
